# Thu thập dữ liệu

Nhóm sẽ thu thập dữ liệu thông tin về xe hơi đã qua sử dụng trên trang web https://www.truecar.com/used-cars-for-sale/listings/

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# Mở Chrome và cho full màn hình
browser = webdriver.Chrome(executable_path='C:/Users/Admin/Desktop/chromedriver')
browser.maximize_window()

In [ ]:
# Mở trang web cần thu thập thông tin
browser.get('https://www.truecar.com/used-cars-for-sale/listings/?sort[]=best_match')

In [ ]:
# Cột bên trái chọn 3 hãng Honda, Ford, Huyndai, Years: 2013-2020 
#- VD: năm 2013: lấy thông tin 100 xe Honda, 100 xe Ford, 100 xe Huyndai
#- Tiếp tục làm vậy đến năm 2020
#--> có tổng cộng (100*3)*8 = 2400 thông tin xe
makes = ['Honda', 'Ford', 'Hyundai','Mercedes-Benz', 'Toyota', 'Mazda']
start_year = 2013
end_year = 2020

In [ ]:
#Chọn năm kết thúc
#year_max = browser.find_elements_by_css_selector('[aria-label="Year Max"] option')
#for year_1 in year_max:
#    if str(start_year) in year_1.text:
#        year_1.click()
#        break

In [ ]:
# chọn năm bắt đầu
#year_min = browser.find_elements_by_css_selector('[aria-label="Year Min"] option')
#for year in year_min:
#    if str(start_year) in year.text:
#        year.click()
#        break

In [ ]:
#Chọn hãng (Make)
#make = browser.find_elements_by_css_selector('[aria-label="Make"] option')
#for i in make:
#    if 'Honda' in i.text:
#        i.click()
#        break

In [ ]:
#browser.quit()

In [ ]:
# Ấn nút sang trang tiếp theo
#browser.find_element_by_css_selector('[data-qa="Pagination-directional-next"]').click()

In [ ]:
##Test thu thập dữ liệu 1 o to
#link = 'https://www.truecar.com/used-cars-for-sale/listing/1FTNE1EL8CDA68267/2012-ford-econoline-cargo-van/'
##link = 'https://www.truecar.com/used-cars-for-sale/listing/2HKRM3H77DH522986/2013-honda-cr-v/?sponsoredVehiclePosition=2'
#text = requests.get(link).text
#cay = BeautifulSoup(text, 'html.parser')
#Model = cay.find('div',{'class':'text-truncate heading-3 margin-right-2 margin-right-sm-3'}).string.split(' ',2)[2]
#print(Model)
#
#
#Vehicle_Overview = cay.find_all('div',{'class':"_1l9na3c align-self-center col"})
#for i in Vehicle_Overview:
#    try:
#        if 'Style' in i.text:
#            Body_style = i.text[5:]
#        elif 'Mileage' in i.text:
#            Mileage = i.text[7:]
#        elif 'Engine' in i.text:
#            Engine = i.text[6:]
#        elif 'Fuel Type' in i.text:
#            Fuel_type = i.text[9:]
#    except:
#        Body_style = ''
#        Mileage = ''
#        Engine = ''
#        Fuel_type = ''
#        continue
#
#print(Body_style)
#
#Year = year
#print(Year)
#
#print(Mileage)
#
#print(Engine)
#
#print(Fuel_type)
#
#try:
#    Number_of_owners = cay.find_all('li',{'class':"_h9wfdq"})[3].text
#except:
#    Number_of_owners =''
#print(Number_of_owners)
#
#try:
#    Price = cay.find('div',{'data-qa':"LabelBlock-text"}).text[1:]
#except:
#    Price = ''
#print(Price)

In [ ]:
web = 'http://truecar.com'
file = open('raw_data.csv', 'w', encoding='utf-8')
file.write(f'make\tmodel\tbody style\tyear\tmileage\tengine\tfuel type\tnumber of owners\tprice\n')
for make in makes:
    #Chọn hãng (Make)
    make_option = browser.find_elements_by_css_selector('[aria-label="Make"] option')
    for make_1 in make_option:
        if make in make_1.text:
            make_1.click()
            break
    time.sleep(5)
    for year in range(start_year,end_year+1):
        dem = 0
        #Chọn năm kết thúc
        year_max = browser.find_elements_by_css_selector('[aria-label="Year Max"] option')
        for year_1 in year_max:
            if str(year) in year_1.text:
                year_1.click()
                break
        time.sleep(5)
        #chọn năm bắt đầu
        year_min = browser.find_elements_by_css_selector('[aria-label="Year Min"] option')
        for year_2 in year_min:
            if str(year) in year_2.text:
                year_2.click()
                break
        time.sleep(5)
        
        html_text = browser.page_source
        tree = BeautifulSoup(html_text, 'html.parser')
        cars = tree.find_all('div', {'data-qa': 'Listings'})
        #lấy thông tin xe
        for car in cars:
            dem=dem+1
            if(dem>100):
                break
            url = car.find('a', {'class':'card card-1 card-shadow card-shadow-hover vehicle-card _1qd1muk'})['href']
            car_html_text = ''
            while car_html_text == '':
                try:
                    car_html_text = requests.get(web+url).text
                    car_tree = BeautifulSoup(car_html_text, 'html.parser')
                    break
                except:
                    print("ConnectionError")
                    time.sleep(5)
                    continue
            #Make
            Make = make
            
            #Model
            try:
                Model = car_tree.find('div',{'class':'text-truncate heading-3 margin-right-2 margin-right-sm-3'}).string.split(' ',2)[2]
            except:
                Model = ''
                
            #Style, Mileage, Engine, Fuel Type    
            Vehicle_Overview = car_tree.find_all('div',{'class':"_1l9na3c align-self-center col"})
            for i in Vehicle_Overview:
                try:
                    if 'Style' in i.text:
                        Body_style = i.text[5:]
                    elif 'Mileage' in i.text:
                        Mileage = i.text[7:]
                    elif 'Engine' in i.text:
                        Engine = i.text[6:]
                    elif 'Fuel Type' in i.text:
                        Fuel_type = i.text[9:]
                except:
                    Body_style = ''
                    Mileage = ''
                    Engine = ''
                    Fuel_type = ''
                    continue
               
            #Year
            Year = year
            
            #Number of owners
            try:
                Number_of_owners = car_tree.find_all('li',{'class':"_h9wfdq"})[3].text
            except:
                Number_of_owners =''

            #Price
            try:
                Price = car_tree.find('div',{'data-qa':"LabelBlock-text"}).text[1:]
            except:
                Price = ''
            file.write(f'{Make}\t{Model}\t{Body_style}\t{Year}\t{Mileage}\t{Engine}\t{Fuel_type}\t{Number_of_owners}\t{Price}\n')
        while True:
            if(dem>100):
                break
            # Ấn nút sang trang tiếp theo
            browser.find_element_by_css_selector('[data-qa="Pagination-directional-next"]').click()
            time.sleep(1)
            
            html_text = browser.page_source
            tree = BeautifulSoup(html_text, 'html.parser')
            cars = tree.find_all('div', {'data-qa': 'Listings'})
            
            #Thông tin xe
            for car in cars:
                dem=dem+1
                if(dem>100):
                    break
                url = car.find('a', {'class':'card card-1 card-shadow card-shadow-hover vehicle-card _1qd1muk'})['href']
                car_html_text = ''
                while car_html_text == '':
                    try:
                        car_html_text = requests.get(web+url).text
                        car_tree = BeautifulSoup(car_html_text, 'html.parser')
                        break
                    except:
                        print("ConnectionError")
                        time.sleep(5)
                        continue
                
                #Make
                Make = make
            
                #Model
                try:
                    Model = car_tree.find('div',{'class':'text-truncate heading-3 margin-right-2 margin-right-sm-3'}).string.split(' ',2)[2]
                except:
                    Model = ''
                    
                #Style, Mileage, Engine, Fuel Type    
                Vehicle_Overview = car_tree.find_all('div',{'class':"_1l9na3c align-self-center col"})
                for i in Vehicle_Overview:
                    try:
                        if 'Style' in i.text:
                            Body_style = i.text[5:]
                        elif 'Mileage' in i.text:
                            Mileage = i.text[7:]
                        elif 'Engine' in i.text:
                            Engine = i.text[6:]
                        elif 'Fuel Type' in i.text:
                            Fuel_type = i.text[9:]
                    except:
                        Body_style = ''
                        Mileage = ''
                        Engine = ''
                        Fuel_type = ''
                        continue
                 
                #Year
                Year = year
                
                #Number of owners
                try:
                    Number_of_owners = car_tree.find_all('li',{'class':"_h9wfdq"})[3].text
                except:
                    Number_of_owners =''
                
                #price
                try:
                    Price = car_tree.find('div',{'data-qa':"LabelBlock-text"}).text[1:]
                except:
                    Price = ''
                file.write(f'{Make}\t{Model}\t{Body_style}\t{Year}\t{Mileage}\t{Engine}\t{Fuel_type}\t{Number_of_owners}\t{Price}\n')
file.close()
browser.quit()
print('Done!')

# Khám phá dữ liệu

In [76]:
import pandas as pd

In [77]:
raw_used_car_df = pd.read_csv('raw_data.csv', sep = '\t')
raw_used_car_df.head()

,make,model,body style,year,mileage,engine,fuel type,number of owners,price
0,Honda,Odyssey,Touring Elite,2013,"122,986",3.5L V-6 Gas,Gas,2.0,"12,895"
1,Honda,Pilot,EX FWD,2013,"80,532",3.5L V-6 Gas,Gas,3.0,"11,694"
2,Honda,CR-V,LX FWD,2013,"79,796",2.4L Inline-4 Gas,Gas,1.0,"13,590"
3,Honda,Civic,LX Sedan Automatic,2013,"76,882",1.8L Inline-4 Gas,Gas,1.0,"7,986"
4,Honda,Civic,LX Sedan Automatic,2013,"113,673",1.8L Inline-4 Gas,Gas,1.0,"6,990"


## Dữ liệu có bao nhiêu dòng, bao nhiêu cột?

In [78]:
raw_used_car_df.shape

(4800, 9)

## Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng ý nghĩa khác nhau không?

### Dữ liệu có các dòng bị lặp không?

In [79]:
raw_used_car_df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
4795    False
4796    False
4797    False
4798    False
4799    False
Length: 4800, dtype: bool

In [80]:
# Bao nhiêu dòng bị lặp?
num_row_dup = raw_used_car_df.duplicated().sum()
num_row_dup

148

In [81]:
# Dòng nào bị lặp?
duplicated_rows = raw_used_car_df[raw_used_car_df.duplicated()]
duplicated_rows

,make,model,body style,year,mileage,engine,fuel type,number of owners,price
393,Honda,Accord,EX-L Sedan I4 CVT,2016,"66,848",2.4L Inline-4 Gas,Gas,2.0,"15,995"
394,Honda,Pilot,EX-L AWD,2016,"124,363",3.5L V-6 Gas,Gas,1.0,"17,792"
395,Honda,Civic,EX Sedan CVT,2016,"79,118",2.0L Inline-4 Gas,Gas,2.0,"13,000"
396,Honda,Civic,LX Sedan CVT,2016,"53,433",2.0L Inline-4 Gas,Gas,1.0,"10,970"
397,Honda,Accord,EX-L V6 Sedan Automatic,2016,"98,855",3.5L V-6 Gas,Gas,1.0,"12,995"
...,...,...,...,...,...,...,...,...,...
4233,Mazda,Mazda3,i Touring 4-Door Automatic,2015,"161,366",2.0L Inline-4 Gas,Gas,1.0,"6,990"
4234,Mazda,CX-5,Grand Touring AWD Automatic,2015,"81,477",2.5L Inline-4 Gas,Gas,1.0,"14,995"
4288,Mazda,CX-5,Grand Touring AWD Automatic,2015,"95,609",2.5L Inline-4 Gas,Gas,1.0,"12,955"
4658,Mazda,Mazda6,Touring Automatic,2019,"29,144",2.5L Inline-4 Gas,Gas,1.0,"19,200"


In [82]:
group_cars = raw_used_car_df.groupby(['make', 'model', 'body style', 'year'])

In [83]:
# Kiểm tra thử dòng 393 xem có đúng là dòng 393 bị lặp không?
# --> đúng, ta thấy dòng 393 bị lặp lại dòng 363 trong bảng dưới
group_cars.get_group(('Honda', 'Accord', 'EX-L Sedan I4 CVT', 2016))

,make,model,body style,year,mileage,engine,fuel type,number of owners,price
332,Honda,Accord,EX-L Sedan I4 CVT,2016,"195,707",2.4L Inline-4 Gas,Gas,1.0,"10,912"
346,Honda,Accord,EX-L Sedan I4 CVT,2016,"35,471",2.4L Inline-4 Gas,Gas,1.0,"15,793"
363,Honda,Accord,EX-L Sedan I4 CVT,2016,"66,848",2.4L Inline-4 Gas,Gas,2.0,"15,995"
378,Honda,Accord,EX-L Sedan I4 CVT,2016,"94,394",2.4L Inline-4 Gas,Gas,1.0,"11,900"
393,Honda,Accord,EX-L Sedan I4 CVT,2016,"66,848",2.4L Inline-4 Gas,Gas,2.0,"15,995"


In [84]:
# Bỏ đi các dòng bị lặp, gán vào dataframe mới
used_car_df = raw_used_car_df.drop_duplicates()

In [88]:
used_car_df.shape

(4652, 9)

In [89]:
# Kiểm tra lại dataframe mới có còn dòng nào bị lặp không?
num_dup = used_car_df.duplicated().sum()
num_dup

0

In [90]:
# Test
assert num_dup == 0
assert used_car_df.shape == (raw_used_car_df.shape[0] - num_row_dup, raw_used_car_df.shape[1])

### Mỗi cột có ý nghĩa gì?

In [64]:
used_car_df.head(1)

,make,model,body style,year,mileage,engine,fuel type,number of owners,price
0,Honda,Odyssey,Touring Elite,2013,"122,986",3.5L V-6 Gas,Gas,2.0,"12,895"


* <b>Cột "make"</b>:             tên hãng xe
* <b>Cột "model"</b>:            tên dòng xe
* <b>Cột "body style"</b>:       hình dáng xe
* <b>Cột "year"</b>:             năm sản xuất chiếc xe
* <b>Cột "mileage"</b>:          tổng số dặm xe đã đi
* <b>Cột "engine"</b>:           thông số động cơ xe
* <b>Cột "fuel type"</b>:        xe đi loại nhiên liệu gì?
* <b>Cột "number of owners"</b>: xe đã qua tay bao nhiêu đời chủ rồi?
* <b>Cột "price"</b>:            giá thành xe hiện được bán

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để xử lý tiếp không?

In [65]:
dtypes = used_car_df.dtypes
dtypes

make                 object
model                object
body style           object
year                  int64
mileage              object
engine               object
fuel type            object
number of owners    float64
price                object
dtype: object

Trong Pandas, kiểu dữ liệu "object" thường ám chỉ chuỗi. Tuy nhiên, "object" có thể ám chỉ một kiểu dữ liệu bất kỳ. "object" giống như con trỏ, có thể trỏ tới một đối tượng bất kỳ

Để biết chính xác kiểu dữ liệu đang được lưu, tiến hành lấy giá trị từng cột để xem

In [66]:
print('make:', type(used_car_df.loc[0, 'make']))
print('model:', type(used_car_df.loc[0, 'model']))
print('body style:', type(used_car_df.loc[0, 'body style']))
print('mileage:', type(used_car_df.loc[0, 'mileage']))
print('engine:', type(used_car_df.loc[0, 'engine']))
print('fuel type:', type(used_car_df.loc[0, 'fuel type']))
print('price:', type(used_car_df.loc[0, 'price']))

make: <class 'str'>
model: <class 'str'>
body style: <class 'str'>
mileage: <class 'str'>
engine: <class 'str'>
fuel type: <class 'str'>
price: <class 'str'>


#### Một vài đánh giá về giá trị cột để tiền xử lý kiểu dữ liệu của cột

* <b>Cột "mileage"</b>, <b>"price"</b>: ta cần dữ liệu dạng số thực, mà ở đây đang ở dạng chuỗi --> tiến hành tiền xử lý để đưa từ kiểu chuỗi sang số thực

# Tiền xử lý kiểu dữ liệu của cột

## Chuyển kiểu dữ liệu cột "mileage", "price" sang kiểu số thực

In [92]:
# bỏ dấu phẩy
used_car_df['mileage'] = used_car_df['mileage'].str.replace(',', '')
used_car_df['price'] = used_car_df['price'].str.replace(',', '')
used_car_df

<ipython-input-92-16ec29b1c9e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_car_df['mileage'] = used_car_df['mileage'].str.replace(',', '')
<ipython-input-92-16ec29b1c9e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_car_df['price'] = used_car_df['price'].str.replace(',', '')


,make,model,body style,year,mileage,engine,fuel type,number of owners,price
0,Honda,Odyssey,Touring Elite,2013,122986,3.5L V-6 Gas,Gas,2.0,12895
1,Honda,Pilot,EX FWD,2013,80532,3.5L V-6 Gas,Gas,3.0,11694
2,Honda,CR-V,LX FWD,2013,79796,2.4L Inline-4 Gas,Gas,1.0,13590
3,Honda,Civic,LX Sedan Automatic,2013,76882,1.8L Inline-4 Gas,Gas,1.0,7986
4,Honda,Civic,LX Sedan Automatic,2013,113673,1.8L Inline-4 Gas,Gas,1.0,6990
...,...,...,...,...,...,...,...,...,...
4795,Mazda,CX-5,Grand Touring FWD,2020,13964,2.5L Inline-4 Gas,Gas,1.0,23888
4796,Mazda,CX-5,Sport FWD,2020,3759,2.5L Inline-4 Gas,Gas,1.0,22498
4797,Mazda,CX-5,Sport FWD,2020,9916,2.5L Inline-4 Gas,Gas,1.0,21990
4798,Mazda,CX-9,Grand Touring AWD,2020,5176,2.5L Inline-4 Gas Turbocharged,Gas,1.0,36992


In [94]:
# chuyển sang kiểu float
used_car_df['mileage'] = used_car_df['mileage'].astype(float)
used_car_df['price'] = used_car_df['price'].astype(float)
used_car_df

<ipython-input-94-bb8d990c0ad0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_car_df['mileage'] = used_car_df['mileage'].astype(float)
<ipython-input-94-bb8d990c0ad0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_car_df['price'] = used_car_df['price'].astype(float)


,make,model,body style,year,mileage,engine,fuel type,number of owners,price
0,Honda,Odyssey,Touring Elite,2013,122986.0,3.5L V-6 Gas,Gas,2.0,12895.0
1,Honda,Pilot,EX FWD,2013,80532.0,3.5L V-6 Gas,Gas,3.0,11694.0
2,Honda,CR-V,LX FWD,2013,79796.0,2.4L Inline-4 Gas,Gas,1.0,13590.0
3,Honda,Civic,LX Sedan Automatic,2013,76882.0,1.8L Inline-4 Gas,Gas,1.0,7986.0
4,Honda,Civic,LX Sedan Automatic,2013,113673.0,1.8L Inline-4 Gas,Gas,1.0,6990.0
...,...,...,...,...,...,...,...,...,...
4795,Mazda,CX-5,Grand Touring FWD,2020,13964.0,2.5L Inline-4 Gas,Gas,1.0,23888.0
4796,Mazda,CX-5,Sport FWD,2020,3759.0,2.5L Inline-4 Gas,Gas,1.0,22498.0
4797,Mazda,CX-5,Sport FWD,2020,9916.0,2.5L Inline-4 Gas,Gas,1.0,21990.0
4798,Mazda,CX-9,Grand Touring AWD,2020,5176.0,2.5L Inline-4 Gas Turbocharged,Gas,1.0,36992.0


In [95]:
new_dtypes = used_car_df.dtypes
new_dtypes

make                 object
model                object
body style           object
year                  int64
mileage             float64
engine               object
fuel type            object
number of owners    float64
price               float64
dtype: object